In [1]:

from enum import IntEnum

class _dataframe_field(IntEnum):
    original_index = 0
    line_flag = 1
    circle_flag = 2
    arc_flag = 3
    point_flag = 4
    start_x = 5
    start_y = 6
    end_x = 7
    end_y = 8
    mid_x = 9
    mid_y = 10
    angle = 11
    length = perimeter = 12
    d_x = 13
    d_y = 14
    u_x = 15
    u_y = 16
    n_x = 17
    n_y = 18
    center_x = 19
    center_y = 20
    radius = 21
    start_angle = 22
    end_angle = 23
    arc_span = 24
    
    @classmethod
    def count(cls) -> int: return len(cls)

class _edge_attribute(IntEnum):
    parallel = 0
    colinear = 1
    perpendicular_distance = 2
    overlap_ratio = 3
    oblique = 4
    intersection_min = 5
    intersection_max = 6
    angle_difference_sin = 7
    angle_difference_cos_min = 8
    angle_difference_cos_max = 9
    
    @classmethod
    def count(cls) -> int: return len(cls)

from typing import Tuple
import math
import torch
from torch import Tensor
from torch import newaxis

@staticmethod
def create_obbs(elements:Tensor, width:float, length_extension:float=0.0) -> Tuple[Tensor, Tensor]:
    
    F = _dataframe_field
    n_elements = elements.size(1)
    
    # Filter supported elements
    is_line = elements[F.line_flag] == 1
    is_circle_or_arc = (elements[F.circle_flag] == 1) | (elements[F.arc_flag] == 1)
    
    filter = is_line | is_circle_or_arc
    
    obbs = torch.empty((n_elements, 4, 2), dtype=torch.float32)
    
    if is_line.any():
        # === LINE ELEMENTS ===
        lines = elements[:, is_line]
        
        # Half dimensions
        half_width = width / 2
        half_length = (lines[F.length] + length_extension) / 2
        
        # Compute displacements
        dx_length = lines[F.u_x] * half_length
        dy_length = lines[F.u_y] * half_length
        dx_width = lines[F.n_x] * half_width
        dy_width = lines[F.n_y] * half_width
        
        # Corners (4 per OBB)
        corner1 = torch.stack([lines[F.mid_x] - dx_length - dx_width,
                                lines[F.mid_y] - dy_length - dy_width], dim=1)
        
        corner2 = torch.stack([lines[F.mid_x] + dx_length - dx_width,
                                lines[F.mid_y] + dy_length - dy_width], dim=1)
        
        corner3 = torch.stack([lines[F.mid_x] + dx_length + dx_width,
                                lines[F.mid_y] + dy_length + dy_width], dim=1)
        
        corner4 = torch.stack([lines[F.mid_x] - dx_length + dx_width,
                                lines[F.mid_y] - dy_length + dy_width], dim=1)
        
        # Stack corners
        obbs[is_line] = torch.stack([corner1, corner2, corner3, corner4], dim=1)
    
    if is_circle_or_arc.any():
        # === ARC ELEMENTS ===
        arcs = elements[:, is_circle_or_arc]
        
        margin = width / 2
        
        u_x, u_y, n_x, n_y = arcs[F.u_x], arcs[F.u_y], arcs[F.n_x], arcs[F.n_y]
        arc_span, radius = arcs[F.arc_span], arcs[F.radius]
        
        dx_length = torch.where(arc_span < torch.pi, n_x * (radius * torch.sin(arc_span/2) + margin), n_x * (radius + margin))
        dy_length = torch.where(arc_span < torch.pi, n_y * (radius * torch.sin(arc_span/2) + margin), n_y * (radius + margin))
        
        dx_width = u_x * (radius * (1 - torch.cos(arc_span/2)) + margin)
        dy_width = u_y * (radius * (1 - torch.cos(arc_span/2)) + margin)
        
        dx_margin = u_x * margin
        dy_margin = u_y * margin
        
        corner1 = torch.stack([arcs[F.mid_x] - dx_length + dx_margin,
                                arcs[F.mid_y] - dy_length + dy_margin], dim=1)
        
        corner2 = torch.stack([arcs[F.mid_x] + dx_length + dx_margin,
                                arcs[F.mid_y] + dy_length + dy_margin], dim=1)
        
        corner3 = torch.stack([arcs[F.mid_x] + dx_length - dx_width,
                                arcs[F.mid_y] + dy_length - dy_width], dim=1)
        
        corner4 = torch.stack([arcs[F.mid_x] - dx_length - dx_width,
                                arcs[F.mid_y] - dy_length - dy_width], dim=1)
        
        obbs[is_circle_or_arc] = torch.stack([corner1, corner2, corner3, corner4], dim=1)
    
    elements = elements[:, filter]
    obbs = obbs[filter]
    
    return elements, obbs # Shape obbs (n_elements, 4, 2)

@staticmethod
def find_overlaping_pairs(elements:Tensor, obbs:Tensor) -> Tuple[Tensor, Tensor]:
    
    F = _dataframe_field
    
    # Compute AABBs (Axis-Aligned Bounding Boxes)
    min_x = obbs[..., 0].min(dim=1).values
    max_x = obbs[..., 0].max(dim=1).values
    min_y = obbs[..., 1].min(dim=1).values
    max_y = obbs[..., 1].max(dim=1).values
    
    # Computing AABB intersection matrix
    mask_x = (min_x[:, newaxis] <= max_x[newaxis, :]) & (min_x[newaxis, :] <= max_x[:, newaxis])
    mask_y = (min_y[:, newaxis] <= max_y[newaxis, :]) & (min_y[newaxis, :] <= max_y[:, newaxis])
    aabb_overlap = mask_x & mask_y
    
    # Candidate pairs
    pairs = torch.argwhere(aabb_overlap) # shape (n, 2)
    i, j = pairs[:, 0], pairs[:, 1]
    
    # Remove self-comparisons and duplicates
    mask = (i < j)
    i, j = i[mask], j[mask]
    
    # Get OBBs for pairs
    obbs_i, obbs_j = obbs[i], obbs[j]
    
    # axes per box
    axes_i = torch.stack([elements[F.u_x, i], elements[F.u_y, i], elements[F.n_x, i], elements[F.n_y, i]], dim=1).reshape(-1,2,2)
    axes_j = torch.stack([elements[F.u_x, j], elements[F.u_y, j], elements[F.n_x, j], elements[F.n_y, j]], dim=1).reshape(-1,2,2)
    axes = torch.cat([axes_i, axes_j], axis=1) # shape (n_pairs, 4, 2)
    
    # Project corners onto axes
    projections_i = torch.einsum('nij,nkj->nik', axes, obbs_i) # Shape (n_pairs, 4, 4)
    projections_j = torch.einsum('nij,nkj->nik', axes, obbs_j)
    
    # Interval comparisons on each axis
    min_i = projections_i.min(dim=2).values
    max_i = projections_i.max(dim=2).values
    min_j = projections_j.min(dim=2).values
    max_j = projections_j.max(dim=2).values
    
    separating_axis = (max_i < min_j) | (max_j < min_i) # True if a separating axis exists
    obb_overlap = ~torch.any(separating_axis, dim=1) # True if overlap
    
    return i[obb_overlap], j[obb_overlap]

@staticmethod
def get_overlap_ratios(lines_a:Tensor, lines_b:Tensor) -> Tuple[Tensor, Tensor, Tensor]:
    
    F = _dataframe_field
    
    # Gather line info
    start_xa, start_ya = lines_a[F.start_x], lines_a[F.start_y]
    u_xa, u_ya = lines_a[F.u_x], lines_a[F.u_y]
    length_a = lines_a[F.length]

    start_xb, start_yb = lines_b[F.start_x], lines_b[F.start_y]
    end_xb, end_yb = lines_b[F.end_x], lines_b[F.end_y]
    mid_xb, mid_yb = lines_b[F.mid_x], lines_b[F.mid_y]
    u_xb, u_yb = lines_b[F.u_x], lines_b[F.u_y]
    length_b = lines_b[F.length]
    
    # Project B's endpoints onto A
    t1 = (start_xb - start_xa) * u_xa + (start_yb - start_ya) * u_ya
    t2 = (end_xb - start_xa) * u_xa + (end_yb - start_ya) * u_ya

    # Get the projected range
    tmin, tmax = torch.minimum(t1, t2), torch.maximum(t1, t2)
    
    # Clip the projection range within [0, length_a]
    overlap_start = torch.clamp(tmin, min=0)
    overlap_end = torch.clamp(tmax, max=length_a)
    overlap_mid = (overlap_start + overlap_end) / 2
    
    # Overlap ratio
    overlap_length = torch.clamp(overlap_end - overlap_start, min=0)
    overlap_a_b = overlap_length / length_a
    overlap_b_a = overlap_length / length_b
    
    # Midpoint of projected overlap on line A
    overlap_mid_xa = start_xa + u_xa * overlap_mid
    overlap_mid_ya = start_ya + u_ya * overlap_mid
    
    # Perpendicular distance to line B
    dx = overlap_mid_xa - mid_xb
    dy = overlap_mid_ya - mid_yb
    
    distance = (dx * -u_yb + dy * u_xb).abs() # From overlaping segment midpoint on line_a to line_b
    
    return overlap_a_b, overlap_b_a, distance # Each: shape (n_pairs,)


In [2]:
import ezdxf

dxf_file = "C:\\Users\\Rafael\\Desktop\\test.dxf"

doc = ezdxf.readfile(dxf_file)
modelSpace = doc.modelspace()

entities = [entity for entity in modelSpace if entity.dxftype() in ('LINE', 'POINT', 'CIRCLE', 'ARC')]

F = _dataframe_field
dataframe = torch.zeros((F.count(), len(entities)), dtype=torch.float32)

for i, entity in enumerate(entities):
    dataframe[F.original_index,i] = i
    entity_type = entity.dxftype()
    
    if entity_type == 'LINE':
        dataframe[F.line_flag,i] = 1
        dataframe[F.start_x,i], dataframe[F.start_y,i], _ = entity.dxf.start
        dataframe[F.end_x,i], dataframe[F.end_y,i], _ = entity.dxf.end
    
    elif entity_type == 'POINT':
        dataframe[F.point_flag,i] = 1
        dataframe[F.mid_x,i], dataframe[F.mid_y,i], _ = entity.dxf.location
        
    elif entity_type in ('CIRCLE', 'ARC'):
        dataframe[F.circle_flag,i] = 1 if entity_type == 'CIRCLE' else 0
        dataframe[F.arc_flag,i] = 1 if entity_type == 'ARC' else 0
        dataframe[F.radius,i] = entity.dxf.radius
        dataframe[F.center_x,i], dataframe[F.center_y,i], _ = entity.dxf.center
        dataframe[F.start_angle,i] = 0 if entity_type == 'CIRCLE' else entity.dxf.start_angle
        dataframe[F.end_angle,i] = 360 if entity_type == 'CIRCLE' else entity.dxf.end_angle
        
# === LINES ===
is_line = dataframe[F.line_flag] == 1

min_length = 1e-3

dataframe[F.mid_x] = torch.where(is_line, (dataframe[F.start_x] + dataframe[F.end_x]) / 2, dataframe[F.mid_x])
dataframe[F.mid_y] = torch.where(is_line, (dataframe[F.start_y] + dataframe[F.end_y]) / 2, dataframe[F.mid_y])

dataframe[F.d_x] = torch.where(is_line, dataframe[F.end_x] - dataframe[F.start_x], dataframe[F.d_x])
dataframe[F.d_y] = torch.where(is_line, dataframe[F.end_y] - dataframe[F.start_y], dataframe[F.d_y])
dataframe[F.angle] = torch.where(is_line, torch.arctan2(dataframe[F.d_y], dataframe[F.d_x]) % (2*torch.pi), dataframe[F.angle])
dataframe[F.length] = torch.where(is_line, torch.sqrt(dataframe[F.d_x]**2 + dataframe[F.d_y]**2), dataframe[F.length])

is_line &= (dataframe[F.length] > min_length)

# Unit direction and unit perpendicular vectors
dataframe[F.u_x] = torch.where(is_line, dataframe[F.d_x] / dataframe[F.length], dataframe[F.u_x])
dataframe[F.u_y] = torch.where(is_line, dataframe[F.d_y] / dataframe[F.length], dataframe[F.u_y])
dataframe[F.n_x] = torch.where(is_line, -dataframe[F.u_y], dataframe[F.n_x])
dataframe[F.n_y] = torch.where(is_line, dataframe[F.u_x], dataframe[F.n_y])

# === POINTS ===
is_point = dataframe[F.point_flag] == 1

for field in [F.start_x, F.end_x]: dataframe[field] = torch.where(is_point, dataframe[F.mid_x], dataframe[field])
for field in [F.start_y, F.end_y]: dataframe[field] = torch.where(is_point, dataframe[F.mid_y], dataframe[field])

is_line_point = (dataframe[F.line_flag] == 1) & (dataframe[F.length] <= min_length)

dataframe[F.point_flag] = torch.where(is_line_point, 1, dataframe[F.point_flag])
dataframe[F.line_flag] = torch.where(is_line_point, 0, dataframe[F.line_flag])

# === CIRCLES AND ARCS ===
is_circle = dataframe[F.circle_flag] == 1
is_arc = dataframe[F.arc_flag] == 1
is_circle_or_arc = is_circle | is_arc

dataframe[F.start_angle] = torch.where(is_circle_or_arc, torch.deg2rad(dataframe[F.start_angle]), dataframe[F.start_angle])
dataframe[F.end_angle] = torch.where(is_circle_or_arc, torch.deg2rad(dataframe[F.end_angle]), dataframe[F.end_angle])

dataframe[F.arc_span] = torch.where(is_circle, 2 * torch.pi, dataframe[F.arc_span])
dataframe[F.arc_span] = torch.where(is_arc, ((dataframe[F.end_angle] - dataframe[F.start_angle]) % (2*torch.pi)), dataframe[F.arc_span])

mid_angle = torch.where(is_circle_or_arc, (dataframe[F.start_angle] + dataframe[F.arc_span] / 2) % (2*torch.pi), 0)

dataframe[F.start_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(dataframe[F.start_angle]), dataframe[F.start_x])
dataframe[F.start_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(dataframe[F.start_angle]), dataframe[F.start_y])
dataframe[F.end_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(dataframe[F.end_angle]), dataframe[F.end_x])
dataframe[F.end_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(dataframe[F.end_angle]), dataframe[F.end_y])
dataframe[F.mid_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(mid_angle), dataframe[F.mid_x])
dataframe[F.mid_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(mid_angle), dataframe[F.mid_y])

dataframe[F.u_x] = torch.where(is_circle_or_arc, (dataframe[F.mid_x] - dataframe[F.center_x]) / dataframe[F.radius], dataframe[F.u_x])
dataframe[F.u_y] = torch.where(is_circle_or_arc, (dataframe[F.mid_y] - dataframe[F.center_y]) / dataframe[F.radius], dataframe[F.u_y])
dataframe[F.n_x] = torch.where(is_circle_or_arc, -dataframe[F.u_y], dataframe[F.n_x])
dataframe[F.n_y] = torch.where(is_circle_or_arc, dataframe[F.u_x], dataframe[F.n_y])

dataframe[F.perimeter] = torch.where(is_circle_or_arc, dataframe[F.radius] * dataframe[F.arc_span], dataframe[F.perimeter])

In [3]:
line_obb_width = 2
parallel_angle_tolerance = 0.02

angle_tolerance = None

In [4]:
obb_width = line_obb_width
angle_tolerance = math.radians(angle_tolerance or parallel_angle_tolerance)

# Compute OBBs
elements, obbs = create_obbs(elements=dataframe, width=obb_width, length_extension=obb_width) # Shape obbs (n_lines, 4, 2)

# Get the pairs of overlapping obbs
i, j = find_overlaping_pairs(elements, obbs)
elements_a, elements_b = elements[:, i], elements[:, j]
'''
# Compute absolute angle difference for line-line pairs in [0, pi]
line_line_pair = (elements_a[F.line_flag] == 1) & (elements_b[F.line_flag] == 1)
angle_difference = torch.where(line_line_pair, torch.abs(elements_a[F.angle] - elements_b[F.angle]), 0)
angle_difference = torch.where(line_line_pair, torch.minimum(angle_difference, torch.pi - angle_difference), angle_difference)

# Keep only pairs with angle difference above threshold
oblique = torch.where(line_line_pair, angle_difference > angle_tolerance, True)
elements_a, elements_b = elements_a[:, oblique], elements_b[:, oblique]
'''

'\n# Compute absolute angle difference for line-line pairs in [0, pi]\nline_line_pair = (elements_a[F.line_flag] == 1) & (elements_b[F.line_flag] == 1)\nangle_difference = torch.where(line_line_pair, torch.abs(elements_a[F.angle] - elements_b[F.angle]), 0)\nangle_difference = torch.where(line_line_pair, torch.minimum(angle_difference, torch.pi - angle_difference), angle_difference)\n\n# Keep only pairs with angle difference above threshold\noblique = torch.where(line_line_pair, angle_difference > angle_tolerance, True)\nelements_a, elements_b = elements_a[:, oblique], elements_b[:, oblique]\n'

In [5]:
n_pairs = elements_a.size(1)

# Filter supported pairs
is_line_a = elements_a[F.line_flag] == 1
is_line_b = elements_b[F.line_flag] == 1
is_arc_a = (elements_a[F.circle_flag] == 1) | (elements_a[F.arc_flag] == 1)
is_arc_b = (elements_b[F.circle_flag] == 1) | (elements_b[F.arc_flag] == 1)

line_line_pair = is_line_a & is_line_b
line_arc_pair, arc_line_pair = is_line_a & is_arc_b, is_arc_a & is_line_b

filter = line_line_pair | (line_arc_pair | arc_line_pair)

In [6]:
intersection_a_min = torch.empty(n_pairs, dtype=torch.float32)
intersection_a_max = torch.empty(n_pairs, dtype=torch.float32)
intersection_b_min = torch.empty(n_pairs, dtype=torch.float32)
intersection_b_max = torch.empty(n_pairs, dtype=torch.float32)

angle_difference_b_a_sin_min = torch.empty(n_pairs, dtype=torch.float32)
angle_difference_b_a_cos_min = torch.empty(n_pairs, dtype=torch.float32)
angle_difference_b_a_sin_max = torch.empty(n_pairs, dtype=torch.float32)
angle_difference_b_a_cos_max = torch.empty(n_pairs, dtype=torch.float32)

In [7]:
filter, line_line_pair

(tensor([True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True]),
 tensor([False, False, False, False, False, False, False,  True, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True,  True]))

In [8]:
lines_a = elements_a[:, line_line_pair]
lines_b = elements_b[:, line_line_pair]

In [9]:
# get angle difference sin and cos
angle_difference_b_a = (lines_b[F.angle] - lines_a[F.angle]) % (2*torch.pi)
angle_difference_b_a

tensor([5.2315, 1.5708, 0.0000])

In [10]:
min_difference = angle_difference_b_a % torch.pi
min_difference = torch.minimum(min_difference, torch.pi - min_difference)
min_difference

tensor([1.0517, 1.5708, 0.0000])

In [11]:
oblique = min_difference >= angle_tolerance
oblique

tensor([ True,  True, False])

In [12]:
lines_a = lines_a[:, oblique]
lines_b = lines_b[:, oblique]
angle_difference_b_a = angle_difference_b_a[oblique]
filter[line_line_pair] = oblique
filter

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False])

In [13]:
line_line_pair

tensor([False, False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False, False,
         True,  True])

In [14]:
line_line_pair &= filter
line_line_pair

tensor([False, False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False, False,
         True, False])

In [15]:
lines_b.size()

torch.Size([25, 2])

In [ ]:
lines_a, lines_b, filter[line_line_pair] = lines_a[:, oblique], lines_b[:, oblique], oblique

In [ ]:
lines_a, lines_b = lines_a[:, oblique], lines_b[:, oblique]
filter[line_line_pair] = oblique